In [23]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error
import keras
from tensorflow import keras as ks
import pickle


In [5]:
def mape(y_true,y_predict):
    return np.sum((np.abs(y_true-y_predict)/y_true)*100)/len(y_true)

In [6]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
feature_df = pd.read_csv('csv_data/first_innings_feature_expanded.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

team_df = pd.read_csv('csv_data/first_innings_team.csv')
opponent_df = pd.read_csv('csv_data/first_innings_opponent.csv')
team_batsman_df = pd.read_csv('csv_data/first_innings_batsman.csv')
team_batsman_max_df = pd.read_csv('csv_data/first_innings_batsman_max.csv')
opponent_bowler_df= pd.read_csv('csv_data/first_innings_opponent_bowler.csv')

#temporary correction code
# team_df['match_id']=feature_df['match_id']
# opponent_df['match_id']=feature_df['match_id']
# team_batsman_df['match_id']=feature_df['match_id']
# opponent_bowler_df['match_id']=feature_df['match_id']




In [7]:
total_df = feature_df.merge(team_df,on='match_id',how='inner',suffixes=('','_team')).\
merge(opponent_df,on='match_id',how='inner',suffixes=('','_opp')).\
merge(team_batsman_df,on='match_id',how='inner',suffixes=('','_bat')).\
merge(opponent_bowler_df,on='match_id',how='inner',suffixes=('','_bow')).\
merge(team_batsman_max_df,on='match_id',how='inner',suffixes=('','_bat_max'))



In [8]:
total_df.shape,feature_df.shape

((760, 56), (760, 20))

In [9]:
total_df['match_id'].nunique(),total_df.shape

(760, (760, 56))

In [10]:
total_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'runs_scored',
       'opponent_base', 'opponent_trend', 'opponent_trend_predict',
       'opponent_mean', 'location_base', 'location_trend',
       'location_trend_predict', 'location_mean', 'current_base',
       'current_trend', 'current_trend_predict', 'current_mean', 'is_train',
       'noise', 'country', 'win_ratio', 'effective_win_by_runs',
       'effective_win_by_wickets', 'matches_played', 'win_count',
       'country_opp', 'win_ratio_opp', 'effective_win_by_runs_opp',
       'effective_win_by_wickets_opp', 'matches_played_opp', 'win_count_opp',
       'total_runs', 'run_rate', 'average_score', 'opponent_variability',
       'player_of_the_match', 'winning_contribution', 'run_rate_effectiveness',
       'negative_rate', 'no_of_wickets', 'wickets_per_match',
       'wickets_per_run', 'no_of_wins', 'opponent_variability_bow',
       'winning_contribution_bow', 'winning_wicket_rate_contribution',
       'batsman', 

In [11]:
#feature_df.dropna(inplace=True)
total_df.isnull().sum()

match_id                             0
match_date                           0
team                                 0
opponent                             0
location                             0
runs_scored                          0
opponent_base                        0
opponent_trend                       0
opponent_trend_predict               0
opponent_mean                        0
location_base                        0
location_trend                       0
location_trend_predict               0
location_mean                        0
current_base                         0
current_trend                        0
current_trend_predict                0
current_mean                         0
is_train                             0
noise                                0
country                              0
win_ratio                            0
effective_win_by_runs                0
effective_win_by_wickets             0
matches_played                       0
win_count                

In [12]:
total_df.dropna(inplace=True)

In [13]:
total_df['match_id'].nunique()

726

In [47]:
import keras as k
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.models import model_from_json

from keras.optimizers import Adam, Adadelta
from keras.regularizers import l2

In [53]:
def create_first_innnings_model(team_vector_len,
                                batsman_vector_length,
                                batsman_max_vector_length,
                                opponent_vector_length,
                                bowler_vector_length,
                                history_vector_length):
    
    team_input = Input((team_vector_len,),name="team_input")
    batsman_input = Input((batsman_vector_length,),name="batsman_input")
    batsman_max_input = Input((batsman_max_vector_length,),name="batsman_max_input")
    opponent_input = Input((opponent_vector_length,),name="opponent_input")
    bowler_input = Input((bowler_vector_length,),name="bowler_input")
    history_input = Input((history_vector_length,),name="history_input")
    
      
    team_output = Dense(team_vector_len * 2,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5),name="team_1")(team_input)
    #team_output = Dense(team_vector_len * 3 , activation='relu',use_bias=True, kernel_initializer='normal',name="team_2")(team_output)
    #team_output = Dense(team_vector_len * 2 , activation='relu',use_bias=True, kernel_initializer='normal',name="team_3")(team_output)
    #team_output = Dense(team_vector_len , activation='relu',use_bias=True, kernel_initializer='normal',name="team_4")(team_output)
    #team_output = Dense(1, name="team_output", activation='relu',use_bias=True, kernel_initializer='normal')(team_output)
    
    batsman_output = Dense(batsman_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5),name="batsman_1")(batsman_input)
    #batsman_output = Dense(batsman_vector_length * 3 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_2")(batsman_output)
    #batsman_output = Dense(batsman_vector_length *2 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_3")(batsman_output)
    #batsman_output = Dense(batsman_vector_length , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_4")(batsman_output)
    #batsman_output = Dense(1, name="batsman_output",  activation='relu',kernel_initializer='normal',use_bias=True)(batsman_output)

    batsman_max_output = Dense(batsman_max_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5),name="batsman_max_1")(batsman_max_input)
    #batsman_max_output = Dense(batsman_max_vector_length * 3 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_max_2")(batsman_max_output)
    #batsman_max_output = Dense(batsman_max_vector_length *2 , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_max_3")(batsman_max_output)
    #batsman_max_output = Dense(batsman_max_vector_length , activation='relu',use_bias=True, kernel_initializer='normal',name="batsman_max_4")(batsman_max_output)
    #batsman_max_output = Dense(1, name="batsman_max_output",  activation='relu',kernel_initializer='normal',use_bias=True)(batsman_max_output)
    

    opponent_output = Dense(opponent_vector_length * 2,activation="relu", kernel_initializer='normal',use_bias=True,bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5),name="opponent_1")(opponent_input)
    #opponent_output = Dense(opponent_vector_length * 3, activation='relu', kernel_initializer='normal',use_bias=True,name="oppoenent_2")(opponent_output)
    #opponent_output = Dense(opponent_vector_length * 2, activation='relu', kernel_initializer='normal',use_bias=True,name="oppoenent_3")(opponent_output)
    #opponent_output = Dense(opponent_vector_length , activation='relu', kernel_initializer='normal',use_bias=True,name="oppoenent_4")(opponent_output)
    #opponent_output = Dense(1, name="opponent_output",activation='relu',  kernel_initializer='normal',use_bias=True)(opponent_output)

    bowler_output = Dense(bowler_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5),name="bowler_1")(bowler_input)
    #bowler_output = Dense(bowler_vector_length *3 , activation='relu',use_bias=True, kernel_initializer='normal',name="bowler_2")(bowler_output)
    #bowler_output = Dense(bowler_vector_length * 2, activation='relu',use_bias=True, kernel_initializer='normal',name="bowler_3")(bowler_output)
    #bowler_output = Dense(bowler_vector_length , activation='relu',use_bias=True, kernel_initializer='normal',name="bowler_4")(bowler_output)
    #bowler_output = Dense(1, name="bowler_output",  activation='relu',kernel_initializer='normal',use_bias=True)(bowler_output)

    history_output = Dense(history_vector_length * 2,activation="relu",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5),name="history_1")(history_input)
    #history_output = Dense(history_vector_length, activation='relu',use_bias=True, kernel_initializer='normal',name="history_2")(history_output)
    #history_output = Dense(1, name="history_output",activation='relu',use_bias=True, kernel_initializer='normal')(history_output)


    runs_output = Concatenate()([team_output, batsman_output,batsman_max_output,opponent_output,bowler_output,history_output])
    #runs_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',name="merged")(runs_output)
    #runs_output = Dense(20,activation="relu",use_bias=True, kernel_initializer='normal',name="merged_2")(runs_output)
    #runs_output = Dense(10,activation="relu",use_bias=True, kernel_initializer='normal',name="merged_3")(runs_output)
    #runs_output = Dense(5,activation="relu",use_bias=True, kernel_initializer='normal',name="merged_4")(runs_output)
    runs_output = Dense(1,name="final_score",use_bias=True, kernel_initializer='normal',bias_regularizer=l2(0.01),kernel_regularizer=l2(0.5))(runs_output)
    
    team_model = Model(inputs=team_input,outputs=team_output)
    batsman_model = Model(inputs=batsman_input,outputs=batsman_output)
    batsman_max_model = Model(inputs=batsman_max_input,outputs=batsman_max_output)
    opponent_model = Model(inputs=opponent_input,outputs=opponent_output)
    bowler_model = Model(inputs=bowler_input,outputs=bowler_output)
    history_model = Model(inputs=history_input,outputs=history_output)
    runs_model = Model(inputs=[team_input,batsman_input,batsman_max_input,opponent_input,bowler_input,history_input],
                      outputs=runs_output)
    
    return team_model,batsman_model,batsman_max_model,opponent_model,bowler_model,history_model,runs_model
    #return runs_model

In [20]:
def load_model(model_name):
    # load json and create model
    json_file = open(model_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_name+".h5")
    return loaded_model

In [21]:
group_encode_model = load_model('group_encode_model')

In [16]:
team_columns = ['win_ratio', 'effective_win_by_runs','effective_win_by_wickets', 'matches_played', 'win_count']

batsman_columns = ['total_runs', 'run_rate', 'average_score', 'opponent_variability','player_of_the_match', 'winning_contribution', 'run_rate_effectiveness']

batsman_max_columns = ['total_runs_bat_max', 'run_rate_bat_max', 'average_score_bat_max', 'opponent_variability_bat_max','player_of_the_match_bat_max', 'winning_contribution_bat_max', 'run_rate_effectiveness_bat_max']

opponent_columns = ['win_ratio_opp', 'effective_win_by_runs_opp','effective_win_by_wickets_opp', 'matches_played_opp', 'win_count_opp']

opponent_bowler_columns = ['negative_rate', 'no_of_wickets', 'wickets_per_match',
       'wickets_per_run', 'no_of_wins', 'opponent_variability_bow',
       'winning_contribution_bow', 'winning_wicket_rate_contribution']

#history_columns = ['opponent_mean','location_mean','current_mean','location_base']
#history_columns = ['location_mean','location_base']


In [24]:
train_total_df = total_df[total_df['is_train']==True]
test_total_df = total_df[total_df['is_train']==False]

enc_map = pickle.load(open('country_location_enc_map.pkl','rb'))
team_loc_list = list(enc_map.keys())

train_total_df=train_total_df[train_total_df['location'].isin(team_loc_list)]
test_total_df=test_total_df[test_total_df['location'].isin(team_loc_list)]






In [32]:
team_scaler = StandardScaler()
batsman_scaler = StandardScaler()
batsman_max_scaler = StandardScaler()
opponent_scaler = StandardScaler()
opponent_bowler_scaler = StandardScaler()
history_scaler = StandardScaler()

team_matrix = team_scaler.fit_transform(np.array(train_total_df[team_columns]))
batsman_matrix = batsman_scaler.fit_transform(np.array(train_total_df[batsman_columns]))
batsman_max_matrix = batsman_max_scaler.fit_transform(np.array(train_total_df[batsman_max_columns]))
opponent_matrix = opponent_scaler.fit_transform(np.array(train_total_df[opponent_columns]))
opponent_bowler_matrix = opponent_bowler_scaler.fit_transform(np.array(train_total_df[opponent_bowler_columns]))
#history_matrix = history_scaler.fit_transform(np.array(train_total_df[history_columns]))

Y = np.array(train_total_df['runs_scored'])

In [33]:

team_oh_list=[]
opponent_oh_list=[]
loc_oh_list=[]
for ind in range(train_total_df.shape[0]):
    team_oh_list.append(enc_map[train_total_df.iloc[ind]['team']])
    opponent_oh_list.append(enc_map[train_total_df.iloc[ind]['opponent']])
    loc_oh_list.append(enc_map[train_total_df.iloc[ind]['location']])

team_oh = np.stack(team_oh_list)
opponent_oh = np.stack(opponent_oh_list)
loc_oh = np.stack(loc_oh_list)

history_matrix_unscaled = group_encode_model.predict([team_oh,opponent_oh,loc_oh])
history_matrix = history_scaler.fit_transform(history_matrix_unscaled)




In [34]:
team_oh_list_test=[]
opponent_oh_list_test=[]
loc_oh_list_test=[]
for ind in range(test_total_df.shape[0]):
    team_oh_list_test.append(enc_map[test_total_df.iloc[ind]['team']])
    opponent_oh_list_test.append(enc_map[test_total_df.iloc[ind]['opponent']])
    loc_oh_list_test.append(enc_map[test_total_df.iloc[ind]['location']])

team_oh_test = np.stack(team_oh_list_test)
opponent_oh_test = np.stack(opponent_oh_list_test)
loc_oh_test = np.stack(loc_oh_list_test)

history_matrix_test_unscaled = group_encode_model.predict([team_oh_test,opponent_oh_test,loc_oh_test])
history_matrix_test = history_scaler.fit_transform(history_matrix_test_unscaled)

In [54]:
team_model,batsman_model,batsman_max_model,opponent_model,opponent_bowler_model,history_model,runs_model = create_first_innnings_model(len(team_columns),
                                                                                                            len(batsman_columns),
                                                                                                            len(batsman_max_columns),
                                                                                                            len(opponent_columns),
                                                                                                            len(opponent_bowler_columns),
                                                                                                            history_matrix_test.shape[1])

# runs_model = create_first_innnings_model(len(team_columns),
#                                          len(batsman_columns), 
#                                          len(batsman_max_columns),
#                                          len(opponent_columns),
#                                          len(opponent_bowler_columns), 
#                                          len(history_columns))
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            

In [55]:
#runs_model.summary()

In [56]:
opt = ks.optimizers.RMSprop(learning_rate=0.0001)
#0.0001

runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_percentage_error","mean_absolute_error"],optimizer=Adam(0.001))
#runs_model.compile(loss="mean_squared_error", metrics=["mean_absolute_percentage_error","mean_absolute_error"],optimizer=opt)



In [57]:


runs_model.fit([team_matrix,batsman_matrix,batsman_max_matrix,opponent_matrix,opponent_bowler_matrix,history_matrix],
               Y, 
               validation_data=(
                   [team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test],
                   Y_test
               ),
               epochs=1000, batch_size=10)
#runs_model.fit([team_matrix,batsman_matrix,batsman_max_matrix,opponent_matrix,opponent_bowler_matrix,history_matrix], Y, epochs=1000, batch_size=5)

Epoch 1/1000
58/58 [==============================] - 0s 7ms/step - loss: 70881.7031 - mean_absolute_percentage_error: 99.6971 - mean_absolute_error: 257.7012 - val_loss: 76587.2109 - val_mean_absolute_percentage_error: 99.2290 - val_mean_absolute_error: 269.1272
Epoch 2/1000
58/58 [==============================] - 0s 3ms/step - loss: 69176.6953 - mean_absolute_percentage_error: 98.3288 - mean_absolute_error: 254.4239 - val_loss: 73669.4688 - val_mean_absolute_percentage_error: 96.9927 - val_mean_absolute_error: 263.6803
Epoch 3/1000
58/58 [==============================] - 0s 4ms/step - loss: 65327.4062 - mean_absolute_percentage_error: 95.2471 - mean_absolute_error: 246.9054 - val_loss: 68255.1875 - val_mean_absolute_percentage_error: 92.6954 - val_mean_absolute_error: 253.2279
Epoch 4/1000
58/58 [==============================] - 0s 6ms/step - loss: 59235.2969 - mean_absolute_percentage_error: 90.0998 - mean_absolute_error: 234.4620 - val_loss: 60759.6211 - val_mean_absolute_percen

58/58 [==============================] - 0s 5ms/step - loss: 5466.1567 - mean_absolute_percentage_error: 26.0249 - mean_absolute_error: 58.2662 - val_loss: 7064.5190 - val_mean_absolute_percentage_error: 30.3345 - val_mean_absolute_error: 64.3548
Epoch 33/1000
58/58 [==============================] - 0s 3ms/step - loss: 5405.1572 - mean_absolute_percentage_error: 25.8903 - mean_absolute_error: 57.8843 - val_loss: 6977.5688 - val_mean_absolute_percentage_error: 30.1382 - val_mean_absolute_error: 63.8050
Epoch 34/1000
58/58 [==============================] - 0s 3ms/step - loss: 5347.2134 - mean_absolute_percentage_error: 25.8489 - mean_absolute_error: 57.4976 - val_loss: 6902.0972 - val_mean_absolute_percentage_error: 29.9740 - val_mean_absolute_error: 63.3818
Epoch 35/1000
58/58 [==============================] - 0s 3ms/step - loss: 5287.3433 - mean_absolute_percentage_error: 25.6455 - mean_absolute_error: 57.1424 - val_loss: 6840.7690 - val_mean_absolute_percentage_error: 29.7966 - val

58/58 [==============================] - 0s 3ms/step - loss: 4159.7686 - mean_absolute_percentage_error: 22.8430 - mean_absolute_error: 50.0025 - val_loss: 5416.8496 - val_mean_absolute_percentage_error: 26.0560 - val_mean_absolute_error: 55.1197
Epoch 64/1000
58/58 [==============================] - 0s 3ms/step - loss: 4132.0005 - mean_absolute_percentage_error: 22.7146 - mean_absolute_error: 49.8317 - val_loss: 5383.6357 - val_mean_absolute_percentage_error: 25.9393 - val_mean_absolute_error: 54.8882
Epoch 65/1000
58/58 [==============================] - 0s 3ms/step - loss: 4101.6187 - mean_absolute_percentage_error: 22.6300 - mean_absolute_error: 49.6796 - val_loss: 5362.9238 - val_mean_absolute_percentage_error: 25.8598 - val_mean_absolute_error: 54.8286
Epoch 66/1000
58/58 [==============================] - 0s 3ms/step - loss: 4075.2512 - mean_absolute_percentage_error: 22.6058 - mean_absolute_error: 49.4657 - val_loss: 5301.8916 - val_mean_absolute_percentage_error: 25.7280 - val

58/58 [==============================] - 0s 4ms/step - loss: 3518.1819 - mean_absolute_percentage_error: 21.0559 - mean_absolute_error: 45.4746 - val_loss: 4637.5747 - val_mean_absolute_percentage_error: 23.7674 - val_mean_absolute_error: 50.3477
Epoch 95/1000
58/58 [==============================] - 0s 4ms/step - loss: 3502.6389 - mean_absolute_percentage_error: 20.9660 - mean_absolute_error: 45.3924 - val_loss: 4641.8989 - val_mean_absolute_percentage_error: 23.7389 - val_mean_absolute_error: 50.4363
Epoch 96/1000
58/58 [==============================] - 0s 3ms/step - loss: 3502.0154 - mean_absolute_percentage_error: 21.0072 - mean_absolute_error: 45.4233 - val_loss: 4626.9268 - val_mean_absolute_percentage_error: 23.7149 - val_mean_absolute_error: 50.3497
Epoch 97/1000
58/58 [==============================] - 0s 3ms/step - loss: 3476.1416 - mean_absolute_percentage_error: 20.8795 - mean_absolute_error: 45.2638 - val_loss: 4602.7310 - val_mean_absolute_percentage_error: 23.6202 - val

58/58 [==============================] - 0s 4ms/step - loss: 3244.8613 - mean_absolute_percentage_error: 20.2236 - mean_absolute_error: 43.4037 - val_loss: 4260.6655 - val_mean_absolute_percentage_error: 22.7286 - val_mean_absolute_error: 48.0975
Epoch 126/1000
58/58 [==============================] - 0s 3ms/step - loss: 3242.7141 - mean_absolute_percentage_error: 20.2116 - mean_absolute_error: 43.4129 - val_loss: 4229.5586 - val_mean_absolute_percentage_error: 22.6899 - val_mean_absolute_error: 47.8408
Epoch 127/1000
58/58 [==============================] - 0s 3ms/step - loss: 3230.0991 - mean_absolute_percentage_error: 20.2148 - mean_absolute_error: 43.3616 - val_loss: 4259.3438 - val_mean_absolute_percentage_error: 22.6947 - val_mean_absolute_error: 48.0753
Epoch 128/1000
58/58 [==============================] - 0s 3ms/step - loss: 3230.1121 - mean_absolute_percentage_error: 20.1400 - mean_absolute_error: 43.3701 - val_loss: 4243.0293 - val_mean_absolute_percentage_error: 22.6403 - 

...

58/58 [==============================] - 0s 3ms/step - loss: 2897.4072 - mean_absolute_percentage_error: 18.7953 - mean_absolute_error: 40.2690 - val_loss: 4028.7285 - val_mean_absolute_percentage_error: 22.0012 - val_mean_absolute_error: 47.5262
Epoch 870/1000
58/58 [==============================] - 0s 3ms/step - loss: 2889.8364 - mean_absolute_percentage_error: 18.8367 - mean_absolute_error: 40.1792 - val_loss: 4045.4587 - val_mean_absolute_percentage_error: 22.0610 - val_mean_absolute_error: 47.6150
Epoch 871/1000
58/58 [==============================] - 0s 3ms/step - loss: 2891.5527 - mean_absolute_percentage_error: 18.8000 - mean_absolute_error: 40.2364 - val_loss: 4053.7996 - val_mean_absolute_percentage_error: 22.0509 - val_mean_absolute_error: 47.6710
Epoch 872/1000
58/58 [==============================] - 0s 3ms/step - loss: 2887.7061 - mean_absolute_percentage_error: 18.8322 - mean_absolute_error: 40.1653 - val_loss: 4060.0703 - val_mean_absolute_percentage_error: 22.0699 - 

58/58 [==============================] - 0s 3ms/step - loss: 2893.9504 - mean_absolute_percentage_error: 18.7914 - mean_absolute_error: 40.1061 - val_loss: 4032.8850 - val_mean_absolute_percentage_error: 22.0234 - val_mean_absolute_error: 47.5912
Epoch 901/1000
58/58 [==============================] - 0s 3ms/step - loss: 2890.5356 - mean_absolute_percentage_error: 18.7836 - mean_absolute_error: 40.2137 - val_loss: 4062.0378 - val_mean_absolute_percentage_error: 22.0867 - val_mean_absolute_error: 47.6186
Epoch 902/1000
58/58 [==============================] - 0s 3ms/step - loss: 2889.6953 - mean_absolute_percentage_error: 18.8360 - mean_absolute_error: 40.2070 - val_loss: 4051.5344 - val_mean_absolute_percentage_error: 22.0400 - val_mean_absolute_error: 47.7231
Epoch 903/1000
58/58 [==============================] - 0s 3ms/step - loss: 2893.4658 - mean_absolute_percentage_error: 18.7694 - mean_absolute_error: 40.2038 - val_loss: 4052.4392 - val_mean_absolute_percentage_error: 22.0573 - 

58/58 [==============================] - 0s 3ms/step - loss: 2891.4363 - mean_absolute_percentage_error: 18.7631 - mean_absolute_error: 40.2627 - val_loss: 4066.1638 - val_mean_absolute_percentage_error: 22.0830 - val_mean_absolute_error: 47.7380
Epoch 932/1000
58/58 [==============================] - 0s 4ms/step - loss: 2887.7949 - mean_absolute_percentage_error: 18.8362 - mean_absolute_error: 40.1787 - val_loss: 4067.5642 - val_mean_absolute_percentage_error: 22.0788 - val_mean_absolute_error: 47.7133
Epoch 933/1000
58/58 [==============================] - 0s 3ms/step - loss: 2892.0347 - mean_absolute_percentage_error: 18.7832 - mean_absolute_error: 40.2226 - val_loss: 4041.0107 - val_mean_absolute_percentage_error: 22.0708 - val_mean_absolute_error: 47.5838
Epoch 934/1000
58/58 [==============================] - 0s 3ms/step - loss: 2889.0376 - mean_absolute_percentage_error: 18.7916 - mean_absolute_error: 40.1794 - val_loss: 4062.9871 - val_mean_absolute_percentage_error: 22.0950 - 

58/58 [==============================] - 0s 4ms/step - loss: 2884.9448 - mean_absolute_percentage_error: 18.7490 - mean_absolute_error: 40.1733 - val_loss: 4059.4792 - val_mean_absolute_percentage_error: 22.1091 - val_mean_absolute_error: 47.7964
Epoch 963/1000
58/58 [==============================] - 0s 3ms/step - loss: 2893.3696 - mean_absolute_percentage_error: 18.7760 - mean_absolute_error: 40.2167 - val_loss: 4071.1914 - val_mean_absolute_percentage_error: 22.0533 - val_mean_absolute_error: 47.8511
Epoch 964/1000
58/58 [==============================] - 0s 3ms/step - loss: 2878.1721 - mean_absolute_percentage_error: 18.6882 - mean_absolute_error: 40.0870 - val_loss: 4057.5952 - val_mean_absolute_percentage_error: 22.1177 - val_mean_absolute_error: 47.7225
Epoch 965/1000
58/58 [==============================] - 0s 6ms/step - loss: 2887.1084 - mean_absolute_percentage_error: 18.8740 - mean_absolute_error: 40.1378 - val_loss: 4052.5103 - val_mean_absolute_percentage_error: 22.0950 - 

58/58 [==============================] - 0s 3ms/step - loss: 2885.8015 - mean_absolute_percentage_error: 18.7422 - mean_absolute_error: 40.1640 - val_loss: 4060.1655 - val_mean_absolute_percentage_error: 22.1063 - val_mean_absolute_error: 47.7787
Epoch 994/1000
58/58 [==============================] - 0s 3ms/step - loss: 2884.2861 - mean_absolute_percentage_error: 18.7996 - mean_absolute_error: 40.1364 - val_loss: 4046.1521 - val_mean_absolute_percentage_error: 22.1329 - val_mean_absolute_error: 47.6266
Epoch 995/1000
58/58 [==============================] - 0s 3ms/step - loss: 2880.6279 - mean_absolute_percentage_error: 18.7069 - mean_absolute_error: 40.1107 - val_loss: 4042.6335 - val_mean_absolute_percentage_error: 22.0737 - val_mean_absolute_error: 47.7199
Epoch 996/1000
58/58 [==============================] - 0s 8ms/step - loss: 2879.0239 - mean_absolute_percentage_error: 18.7422 - mean_absolute_error: 40.0662 - val_loss: 4063.7590 - val_mean_absolute_percentage_error: 22.0974 - 

In [39]:
team_matrix_test = team_scaler.transform(np.array(test_total_df[team_columns]))
batsman_matrix_test = batsman_scaler.transform(np.array(test_total_df[batsman_columns]))
batsman_max_matrix_test = batsman_max_scaler.transform(np.array(test_total_df[batsman_max_columns]))
opponent_matrix_test = opponent_scaler.transform(np.array(test_total_df[opponent_columns]))
opponent_bowler_matrix_test = opponent_bowler_scaler.transform(np.array(test_total_df[opponent_bowler_columns]))
#history_matrix_test = history_scaler.transform(np.array(test_total_df[history_columns]))

Y_test = np.array(test_total_df['runs_scored'])

In [40]:
runs_model.evaluate([team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test],Y_test)


5/5 [==============================] - 0s 1ms/step - loss: 4174.1196 - mean_absolute_percentage_error: 22.6665 - mean_absolute_error: 49.8182


[4174.11962890625, 22.666494369506836, 49.81822967529297]

In [32]:
runs_model.predict([team_matrix_test,batsman_matrix_test,batsman_max_matrix_test,opponent_matrix_test,opponent_bowler_matrix_test,history_matrix_test])


array([[250.78212],
       [261.64954],
       [259.36664],
       [238.9258 ],
       [244.84506],
       [239.0445 ],
       [251.80794],
       [301.1614 ],
       [273.16183],
       [304.29764],
       [266.0955 ],
       [230.72116],
       [252.98524],
       [264.87933],
       [263.2293 ],
       [248.97499],
       [246.4898 ],
       [240.80481],
       [273.53183],
       [305.63452],
       [301.8208 ],
       [256.0221 ],
       [281.68826],
       [308.81088],
       [223.2198 ],
       [286.88297],
       [241.20824],
       [269.76672],
       [253.94502],
       [254.89374],
       [279.98004],
       [247.9804 ],
       [222.05179],
       [240.52562],
       [249.3945 ],
       [265.87332],
       [240.79692],
       [256.15927],
       [264.3823 ],
       [233.71213],
       [240.28232],
       [316.07184],
       [239.87456],
       [264.12967],
       [307.2731 ],
       [248.04909],
       [246.59392],
       [257.59677],
       [232.9329 ],
       [274.2479 ],


In [185]:
# # serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

In [186]:
def store_model(model,name):
    # serialize model to JSON
    model_json = model.to_json()
    with open(name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(name+".h5")
    print("Saved model to disk")

In [187]:
store_model(runs_model,'run_model')
store_model(team_model,'team_model')
store_model(batsman_model,'batsman_model')
store_model(opponent_model,'opponent_model')
store_model(bowler_model,'bowler_model')
store_model(history_model,'history_model')

Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk


In [188]:
import pickle
# batsman_scaler = StandardScaler()
# opponent_scaler = StandardScaler()
# opponent_bowler_scaler = StandardScaler()
# history_scaler = StandardScaler()
pickle.dump(batsman_scaler,open('batsman_scaler.pkl','wb'))
pickle.dump(opponent_scaler,open('opponent_scaler.pkl','wb'))
pickle.dump(opponent_bowler_scaler,open('opponent_bowler_scaler.pkl','wb'))
pickle.dump(history_scaler,open('history_scaler.pkl','wb'))

In [194]:
import pickle
batsman_performance_df = pickle.load(open('batsman_performance.pkl','rb'))

In [195]:
performance_matrix = batsman_scaler.transform(np.array(batsman_performance_df[batsman_columns]))

In [196]:
performance_matrix[0,:]

array([-0.73848173,  1.33302935,  0.78490317, -0.69039957, -1.47933909,
        5.67791726,  1.04933019])

In [197]:
batting_score = batsman_model.predict(performance_matrix)

In [198]:
batsman_performance_df['score']=batting_score

In [199]:
#batsman_performance_df['score_2']=score_2

In [200]:
batsman_performance_df=batsman_performance_df.sort_values('score',ascending=False)

In [201]:
batsman_performance_df.head(20)

,batsman,country,total_runs,run_rate,average_score,opponent_variability,player_of_the_match,winning_contribution,run_rate_effectiveness,score
143,V Kohli,India,970,0.992835,80.833333,3,4,0.233001,1.081033,108.672569
50,AJ Tye,Australia,57,1.187500,8.142857,1,0,0.007264,3.769976,108.229439
23,LRPL Taylor,New Zealand,582,0.859675,72.750000,2,2,0.297794,1.011823,89.851936
141,RG Sharma,India,1021,0.997070,60.058824,6,2,0.310667,1.082522,87.897491
201,TP Ura,Papua New Guinea,237,0.955645,79.000000,3,1,0.251282,1.252151,87.282768
278,CJ Dala,South Africa,5,1.666667,5.000000,1,0,0.000000,0.000000,83.329201
51,JJ Roy,England,787,1.049333,46.294118,4,3,0.165285,1.107739,81.920570
80,TB Sole,Scotland,20,1.333333,20.000000,1,1,0.000000,0.000000,81.580238
3,Rameez Shahzad,United Arab Emirates,405,0.858051,67.500000,4,1,0.294340,1.072237,81.487778
52,JM Bairstow,England,869,1.152520,45.736842,4,1,0.173072,1.189467,80.563171


In [220]:
batsman_performance_df.sort_values('winning_contribution',ascending=False)

,batsman,country,total_runs,run_rate,average_score,opponent_variability,player_of_the_match,winning_contribution,run_rate_effectiveness,score
3,Rameez Shahzad,United Arab Emirates,405,0.858051,67.500000,4,1,0.294340,1.072237,0.958123
79,CS MacLeod,Scotland,372,0.963731,46.500000,4,1,0.279018,1.181252,0.958120
31,Fakhar Zaman,Pakistan,728,0.962963,48.533333,6,2,0.276699,1.182598,0.958124
77,MH Cross,Scotland,322,0.838542,40.250000,4,2,0.268973,1.003066,0.958115
61,BRM Taylor,Zimbabwe,839,0.874870,44.157895,7,1,0.261538,1.097580,0.958122
...,...,...,...,...,...,...,...,...,...,...
354,SM Kavanagh,Ireland,18,0.450000,18.000000,1,0,0.000000,0.000000,0.958089
424,RK Paudel,Nepal,9,0.642857,9.000000,1,0,0.000000,0.000000,0.958089
380,CJ Chibhabha,Zimbabwe,43,0.505882,14.333333,1,0,0.000000,0.000000,0.958089
253,Ehsan Khan,Hong Kong,58,0.491525,11.600000,5,0,0.000000,0.000000,0.958089


In [221]:
batsman_performance_df[batsman_performance_df['country']=='Australia']

,batsman,country,total_runs,run_rate,average_score,opponent_variability,player_of_the_match,winning_contribution,run_rate_effectiveness,score
362,SE Marsh,Australia,416,0.997602,59.428571,2,0,0.015110,0.950761,0.958120
271,AJ Healy,Australia,170,1.062500,42.500000,2,0,0.116758,1.193853,0.958117
50,AJ Tye,Australia,57,1.187500,8.142857,1,0,0.002060,3.370879,0.958116
276,A Gardner,Australia,70,1.372549,23.333333,2,1,0.048077,1.542232,0.958116
272,MM Lanning,Australia,195,0.898618,48.750000,2,1,0.133929,1.009710,0.958116
275,BL Mooney,Australia,105,1.153846,35.000000,2,0,0.072115,1.296492,0.958116
270,NE Bolton,Australia,202,0.863248,50.500000,2,2,0.138736,0.969968,0.958116
277,NJ Carey,Australia,29,1.611111,9.666667,2,0,0.019918,1.810287,0.958116
42,AJ Finch,Australia,493,0.880357,44.818182,2,1,0.028159,0.731249,0.958114
46,MR Marsh,Australia,186,0.845455,37.200000,1,0,0.021978,1.198535,0.958112


In [222]:
analysis_df = pd.DataFrame()
analysis_df['team']=team_model.predict(team_matrix).reshape(-1)
analysis_df['batsman']=batsman_model.predict(batsman_matrix).reshape(-1)
analysis_df['opponent']=opponent_model.predict(opponent_matrix).reshape(-1)
analysis_df['opponent_bowler']=bowler_model.predict(opponent_bowler_matrix).reshape(-1)
analysis_df['history']=history_model.predict(history_matrix).reshape(-1)

In [223]:
analysis_df['team'].std(),analysis_df['batsman'].std(),analysis_df['opponent'].std(),analysis_df['opponent_bowler'].std(),analysis_df['history'].std()



(9.837326615524944e-06,
 7.87309545557946e-06,
 5.939106358709978e-06,
 8.146505024342332e-06,
 0.0010397277073934674)

In [224]:
runs_model.predict([team_matrix,batsman_matrix,opponent_matrix,opponent_bowler_matrix,history_matrix])

array([[240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
       [240.29724],
